# Text Analysis on sydneysiege data

In [21]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [23]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,0
7,544268900351766528,16215478,#BREAKING A police operation is underway in th...,Sun Dec 14 23:13:12 +0000 2014,0
15,544274797640044544,7587032,Australian Police Operation Under Way In Sydne...,Sun Dec 14 23:36:38 +0000 2014,0
25,544275574642835456,27964284,Prayers needed for the situation happening in ...,Sun Dec 14 23:39:43 +0000 2014,0
47,544275733607366656,374712154,This is not going to end well in Sydney . Wake...,Sun Dec 14 23:40:21 +0000 2014,0
...,...,...,...,...,...
22682,544520042810200064,442653867,"#sydneysiege is over. 2 confirmed dead, #PrayF...",Mon Dec 15 15:51:09 +0000 2014,1
22684,544520273718812672,100629223,Police confirm that #sydneysiege is finally ov...,Mon Dec 15 15:52:04 +0000 2014,1
22702,544520932405284864,7587032,Seven people reportedly taken away on stretche...,Mon Dec 15 15:54:41 +0000 2014,1
22736,544521141335764992,342894187,BREAKING: CNN: Sydney police storm cafe. 2 con...,Mon Dec 15 15:55:31 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [36]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
          
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [37]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  @SkyNews Al-nusra front https://t.co/ElpNQ129ER …
cleaned text:  alnusra front


### applied on original data (**df**)

In [38]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [39]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,polic oper underway martin place sydney cbd pe...,nswpolic polic oper underway martin place sydn...
1,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544273361334194176,14875436,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:30:56 +0000 2014,0,polic oper underway martin place sydney cbd pe...,rt polic oper underway martin place sydney cbd...
2,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544275366588588032,14121167,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:38:54 +0000 2014,0,polic oper underway martin place sydney cbd pe...,nswpolic polic oper underway martin place sydn...
3,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544276615316770816,187877777,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:43:51 +0000 2014,0,polic oper underway martin place sydney cbd pe...,rt polic oper underway martin place sydney cbd...
4,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544278115589976065,372216479,RT “@nswpolice: A police operation is underway...,Sun Dec 14 23:49:49 +0000 2014,0,polic oper underway martin place sydney cbd pe...,rt nswpolic polic oper underway martin place s...


### applied on source tweet data (**src_tw_df**) 

In [40]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [41]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [42]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [43]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [44]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,polic oper underway martin place sydney cbd pe...,nswpolic polic oper underway martin place sydn...,Negative,Negative


### applied on source tweet data only 

In [45]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/sydneysiege/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/sydneysiege/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/sydneysiege/senti_source.pdf',dpi=300)

### save data to csv 

In [13]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)

In [ ]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [46]:
import text2emotion as te

In [47]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [52]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emotion']=emo

In [53]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)

In [54]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emotion']=emo1

In [55]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)